# Create anonymized data

For examples and sharing, we create an anonymized dataset.

## Data requirements

- Sewer mains
  * Sewer gravity mains and possibly force mains (as one vector).
  * We assume the mains to be already imported in GRASS GIS and we do not export at the end (this can be added).

<div class="alert alert-block alert-warning">
<b>Note:</b>
    An addon tool for GRASS GIS, <em>v.rast.move</em>, is needed. Download it using <code>g.extension v.rast.move</code>.
</div>

## Software setup 

We will use a couple of standard Python packages and GRASS GIS.

For now, the setup here assumes Linux. Instructions for Windows are available at [GRASS GIS Jupyter notebooks wiki page](https://grasswiki.osgeo.org/wiki/GRASS_GIS_Jupyter_notebooks#Running_a_Jupyter_notebook_locally).

In [ ]:
# Import Python standard library and IPython packages we need.
import os
import subprocess
import sys
import json
from pathlib import Path

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

## Get the data ready

This assumes we have usable sewer mains already imported in GRASS GIS as `raleigh_gravity_mains@imported_RaleighSewer` in location `data/ww_covid_2022/`.

In [ ]:
grass_project = "data/ww_covid_2022/"
subproject = "anonymized_data"

In [ ]:
!grass -e -c $grass_project/$subproject

In [ ]:
session = gj.init(Path(grass_project) / subproject)

In [ ]:
gs.run_command(
    "v.extract",
    input="raleigh_gravity_mains@imported_RaleighSewer",
    where="Discharge =  'Smith Creek'",
    output="mains_subset",
)

## Create dense lines

First create continuos polylines (multi-lines) from separate lines for better handling further on and then densify them by splitting long straight lines into multi-lines.

In [ ]:
gs.run_command(
    "v.build.polylines",
    input="mains_subset",
    output="mains_subset_polylines",
    cats="first",
    type="line,boundary",
)
gs.run_command(
    "v.split",
    input="mains_subset_polylines",
    layer="-1",
    output="mains_subset_dense",
    length=75,
    units="map",
)

## Compute kernel density

In [ ]:
gs.run_command("g.region", vector="mains_subset_dense", res=100, flags="a", grow=1)

In [ ]:
%%bash
# This would be better without bash, the ASCII round trip and with densified lines.
v.out.ascii mains_subset separator=" " type=line format=standard | grep -vE "^L|^ 1" | grep -v "[a-zA-Z]" | v.in.ascii input=- output=mains_points format=point separator=" " x=2 y=3 --o

In [ ]:
gs.run_command(
    "v.kernel",
    input="mains_points",
    output="mains_density",
    radius=1000,
    dsize=0.0,
    segmax=100.0,
    distmax=100.0,
    multiplier=1000,
    kernel="gaussian",
)

## Perform the move

In [ ]:
gs.mapcalc(
    "mains_density_randomized = 2 * rand(mains_density, 1 / mains_density)"
    " + 1000 * sin(row() / 100)"
    " + 100 * sin(col())",
    seed=1,
)
gs.run_command("r.colors", map="mains_density_randomized", color="difference")

In [ ]:
gs.run_command(
    "v.rast.move",
    input="mains_subset_dense",
    x_raster="mains_density_randomized",
    y_raster="mains_density_randomized",
    output="mains_subset_moved",
)

In [ ]:
sewer_map = gj.Map(use_region=True, width=900)
sewer_map.d_background(color="white")
sewer_map.d_vect(
    map="mains_subset_dense", color="blue", width=1, legend_label="Original"
)
sewer_map.d_vect(map="mains_subset_moved", color="red", width=1, legend_label="Shifted")
sewer_map.d_barscale(flags="n", at=(65, 4), length=2000)
sewer_map.d_legend_vect(at=(85, 13), title="Sewer mains")
sewer_map.show()

In [ ]:
sewer_map = gj.Map(use_region=True, width=900)
sewer_map.d_background(color="white")
sewer_map.d_rast(map="mains_density_randomized")
sewer_map.d_vect(map="mains_subset_moved", color="gray", width=1, legend_label="Mains")
sewer_map.d_legend_vect(at=(70, 10))
sewer_map.d_legend(
    raster="mains_density_randomized", range=(-100, 100), at=(5, 25, 85, 90)
)
sewer_map.show()

## Attributes

During the process, we dropped the attribute table, so now there is no link to the (attribute) database and thus no attributes:

In [ ]:
%%bash
v.info -e mains_subset_moved | grep num_dblinks